<a href="https://colab.research.google.com/github/shavindukesara/Telco-Churn-Predictor/blob/main/models/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

In [3]:
def create_nn_model(learning_rate=0.001, dropout_rate=0.3):
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        layers.Dropout(dropout_rate),
        layers.Dense(64, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', keras.metrics.AUC(),
                 keras.metrics.Precision(), keras.metrics.Recall()]
    )
    return model

In [4]:
# Hyperparameter tuning
def tune_nn_hyperparameters(X_train, y_train):
    model = KerasClassifier(model=create_nn_model, verbose=0)

    param_grid = {
        'model__learning_rate': [0.001, 0.01],
        'model__dropout_rate': [0.2, 0.3, 0.5],
        'batch_size': [32, 64],
        'epochs': [50, 100]
    }

    grid = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=3,
        scoring='f1',
        n_jobs=-1
    )

    grid_result = grid.fit(X_train, y_train)
    return grid_result.best_params_